In [1]:
import pandas as pd
import sqlalchemy
from sqlalchemy import *

In [2]:
engine = create_engine("sqlite:///data/movies-db.sqlite")
conn = engine.connect()

In [14]:
df = pd.read_csv("data/genome-scores.csv")
df2 = pd.read_csv("data/genome-tags.csv")
df3 = pd.read_csv("data/links.csv")
df4 = pd.read_csv("data/movies.csv")
df5 = pd.read_csv("data/ratings.csv")
genreMatrix = pd.read_csv('clean_data/genreMatrix.csv')
links = pd.read_csv('links.csv')
other_info = pd.read_csv('clean_data/imdb_details_clean.csv')

In [68]:
my_tags = pd.read_csv('data/my_tags.csv').rename(columns={'imdbID_formatted':'imdbID'})

In [7]:
ratings = df5.groupby('movieId').mean().drop(columns = ['userId', 'timestamp'])

In [16]:
ids = links.drop(columns=['imdbId', 'tmdbId']).rename(columns={'imdbID_formatted':'imdbID'})

In [67]:
movies = pd.merge(ids, df4, how='inner', on='movieId')
movies.head()

,movieId,imdbID,title,genres
0,1,tt0114709,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,tt0113497,Jumanji (1995),Adventure|Children|Fantasy
2,3,tt0113228,Grumpier Old Men (1995),Comedy|Romance
3,4,tt0114885,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,tt0113041,Father of the Bride Part II (1995),Comedy


In [20]:
movies_ratings = pd.merge(movies, ratings, how='inner', on='movieId')

In [28]:
movies_genres_ratings = pd.merge(movies_ratings, genreMatrix, how='inner', on='imdbID')

In [42]:
rated = pd.get_dummies(other_info['Rated'])
rated['imdbID'] = other_info['imdbID']

In [70]:
ratings_genres_tags = pd.merge(movies_genres_ratings, my_tags, how='inner', on='imdbID')

In [71]:
ratings_genres_tags.to_csv('data/movies_my_tags.csv')

In [ ]:
tags_df = df.pivot(index='movieId', columns='tagId', values='relevance')

In [ ]:
movies_tags_ratings = pd.merge(movies_ratings, tags_df, how='inner', on='movieId')

In [72]:
movies_genres_ratings.to_sql('movies_genres', con=conn)